In [1]:
from transitions import Machine

from transitions.extensions.asyncio import AsyncTimeout, AsyncMachine

import enum 
import logging
import asyncio

from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
from pprint import pprint

from datetime import datetime
from math import floor, ceil
import time
from time import sleep

import sys
import os

import zmq
import zmq.asyncio
# from s import Context
import json

logging.basicConfig(level=logging.INFO)

import ccxt
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
client = MongoClient()
db=client.fsmbot
# trades = {}
exchange = getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})



In [ ]:
# exchange.fetchOrderStatus('686722117', symbol='ALGOUSDT')
exchange.fetchOpenOrders(symbol='ALGOUSDT')

In [36]:
exchange.fetchOrderStatus('686722117', symbol='ALGOUSDT')
# exchange.fetchOrders(symbol='ALGOUSDT')

'closed'

In [ ]:
# test ticker zmq publisher
zContext = zmq.asyncio.Context()

socket = zContext.socket(zmq.SUB)
socket.connect("tcp://localhost:5556")
socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

while True:
    tickerString = await socket.recv_string()
    _, symbol, ticker = tickerString.split()
    ticker = json.loads(ticker)
    if symbol == 'BNBUSDT':
        print(ticker)


In [4]:
try:
    raise Exception('BOOOOM')
except BaseException as err:
    print(err)


BOOOOM


In [5]:
(504 - 471) * 83.33

2749.89